<a href="https://colab.research.google.com/github/Gajeshgif/Gajesh/blob/main/Heart_Disease_Prediction_%7C_Classification_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
fedesoriano_heart_failure_prediction_path = kagglehub.dataset_download('fedesoriano/heart-failure-prediction')

print('Data source import complete.')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
cat_cols=["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"]
num_cols=["Age","RestingBP","Cholesterol","FastingBS","MaxHR","Oldpeak"]

In [ ]:
for i in cat_cols:
    print(df[i].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for i in num_cols:
    plt.figure()
    sns.boxplot(x=df[i])
    plt.title("Outlier Detection")
    plt.show()

In [ ]:
df["HeartDisease"].value_counts()

In [ ]:
df.head()

In [ ]:
cat_cols=["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"]
num_cols=["Age","RestingBP","Cholesterol","MaxHR","Oldpeak"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

X=df.drop("HeartDisease",axis=1)
y=df["HeartDisease"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols),
        ("cat", OneHotEncoder(), cat_cols)
    ]
)

X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def test_classification_models(X_train, X_test, y_train, y_test):
    models = {
        'LogisticRegression': LogisticRegression(max_iter=1000),
        'DecisionTree': DecisionTreeClassifier(),
        'RandomForest': RandomForestClassifier(),
        'GradientBoosting': GradientBoostingClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'SVC': SVC(probability=True),
        'KNN': KNeighborsClassifier(),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)
    }

    results = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        results.append({
            'Model': name,
            'Accuracy': accuracy_score(y_test, preds),
            'Precision': precision_score(y_test, preds, average='weighted', zero_division=0),
            'Recall': recall_score(y_test, preds, average='weighted', zero_division=0),
            'F1-Score': f1_score(y_test, preds, average='weighted')
        })

    return pd.DataFrame(results).sort_values(by='F1-Score', ascending=False).reset_index(drop=True)



In [ ]:
df_results = test_classification_models(X_train_scaled, X_test_scaled, y_train, y_test)
print(df_results)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
X=preprocessor.fit_transform(X)

In [ ]:
model=RandomForestClassifier()
print(cross_val_score(model, X, y, cv=5))